In [1]:
!pip -q install rank_bm25
!pip -q install pyvi
!pip -q install vncorenlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 66.8 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 27.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done


In [2]:
# from vncorenlp import VnCoreNLP
# rdrsegmenter = VnCoreNLP("/kaggle/input/vncorenlp/other/default/1/VnCoreNLP/VnCoreNLP-1.2.jar", annotators="wseg")

In [3]:
!pip install py_vncorenlp
import py_vncorenlp

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.6 MB/s eta 0:00:0000:01
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4305 sha256=93119fe7cec6ef7276c3603dcdb8ffa177821212eeb486295a4774cfea9d3c18
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


In [4]:
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/kaggle/input/vncorenlp/other/default/1/VnCoreNLP')

In [5]:
import os
import json
import pandas as pd
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import nltk
from pyvi.ViTokenizer import tokenize

In [6]:
# Đường dẫn tới thư mục chứa các file JSON
qa_folder_path = '/kaggle/input/all-dataset-qa-retrieve/all_dataset_qa_retrieve'

# Danh sách để chứa dữ liệu
qa_data = []

# Lặp qua tất cả các file trong thư mục và sử dụng tqdm để theo dõi tiến trình
for file_name in tqdm(os.listdir(qa_folder_path), desc="Reading files"):
    
    if file_name.endswith('.json'):  # Kiểm tra nếu file là file JSON
        file_path = os.path.join(qa_folder_path, file_name)
        
        # Mở và đọc dữ liệu từ file JSON
        with open(file_path, 'r', encoding='utf-8') as f:
            qa_file_data = json.load(f)
            
            # Lặp qua các câu hỏi và câu trả lời trong dữ liệu
            for qa in qa_file_data['qa']:
                qa_data.append({
                    'question': qa['question'],
                    'answer': qa['answer'],
                    'id': qa['id']
                })

Reading files: 100%|██████████| 499/499 [00:02<00:00, 175.93it/s]


In [7]:
# Tạo DataFrame từ dữ liệu đã thu thập
qa_df = pd.DataFrame(qa_data)

In [8]:
qa_df['tokenized_question'] = qa_df['question'].apply(lambda x: tokenize(x).lower().split())

In [9]:
qa_df['segmented'] = qa_df['question'].apply(lambda x: rdrsegmenter.word_segment(x))

In [10]:
qa_df['segmented'] =  qa_df['segmented'].apply(lambda x: ' '.join(x))

In [11]:
import pickle

# Load lại bm25 model
with open('/kaggle/input/bm25pyvi/other/default/1/bm25_model_pyvi.pkl', 'rb') as f:
    bm25 = pickle.load(f)

In [12]:
chunked_df = pd.read_json('/kaggle/input/df-full-and-chunked/df/chunked_df.json')

In [ ]:
chunked_df['segmented'] = chunked_df['content'].apply(lambda x: rdrsegmenter.word_segment(x))

In [ ]:
chunked_df['segmented'] = chunked_df['segmented'].apply(lambda x: ' '.join(x))

In [ ]:
from transformers import AutoTokenizer
!pip install -q sentence-transformers
from sentence_transformers import CrossEncoder

MODEL_ID = 'itdainb/PhoRanker'
MAX_LENGTH = 512

model = CrossEncoder(MODEL_ID, max_length=MAX_LENGTH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

def truncate_text(text, max_length=256):
    tokens = tokenizer.encode(text, truncation=True, max_length=max_length, add_special_tokens=True)
    return tokenizer.decode(tokens, skip_special_tokens=True)

chunked_df['segmented'] = chunked_df['segmented'].apply(lambda x: truncate_text(x, max_length=MAX_LENGTH))

In [ ]:
import logging
from transformers import logging as transformers_logging

# Tắt log từ transformers
logging.getLogger("transformers").setLevel(logging.ERROR)
transformers_logging.set_verbosity_error()


In [ ]:
import os
import sys
import numpy as np
from tqdm import tqdm
from sentence_transformers import CrossEncoder
from contextlib import contextmanager, redirect_stdout, redirect_stderr

@contextmanager
def suppress_output():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        try:
            sys.stdout = devnull
            sys.stderr = devnull
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr

def evaluate_bm25_with_crossencoder_top_k(df, qa_df, bm25, top_k_bm25=200, top_k_values=list(range(1, 101))):
    results = {f"Top-{k}": 0 for k in top_k_values}  # Khởi tạo kết quả đánh giá
    total_queries = len(qa_df)

    for _, row in tqdm(qa_df.iterrows(), total=total_queries):
        query = row['tokenized_question']
        true_id = row['id']

        # Step 1: Dùng BM25 lấy Top-200
        scores = bm25.get_scores(query)
        top_indices = scores.argsort()[::-1][:top_k_bm25]  # Top-200 kết quả từ BM25
        top_candidates = df.iloc[top_indices]

        # Step 2: Tạo input cho CrossEncoder
        crossencoder_inputs = [[row['segmented'], chunk] for chunk in top_candidates['segmented']]

        # Step 3: Tính điểm bằng CrossEncoder với suppress_output để tắt hiển thị
        with suppress_output():
            crossencoder_scores = model.predict(crossencoder_inputs)

        # Step 4: Chọn Top-100 từ CrossEncoder
        cross_top_indices = np.argsort(crossencoder_scores)[::-1][:len(top_k_values)]
        top_ids = top_candidates.iloc[cross_top_indices]['id'].tolist()

        # Step 5: Đánh giá cho từng Top-k
        for k in top_k_values:
            if true_id in top_ids[:k]:
                results[f"Top-{k}"] += 1

    # Tính độ chính xác cho mỗi Top-k
    for k in top_k_values:
        results[f"Top-{k}"] /= total_queries

    return results

top_k_values = list(range(1, 101))  # Top-1 đến Top-100
evaluation_results = evaluate_bm25_with_crossencoder_top_k(chunked_df, qa_df, bm25, top_k_bm25=200, top_k_values=top_k_values)


In [ ]:
# In kết quả đánh giá
for k, v in evaluation_results.items():
    print(f"{k}: {v * 100:.2f}%")